# TV@J 2025 - Měření Magnetického Pole a Polohy Plazmatu

## Měření Magnetického Pole
K určení magnetického pole lze využít **Mirnovových cívek (MC)** řadících se mezi tzv. pasivní magnetické diagnostiky. To znamená, že zaznamenávají napětí indukované změnou magnetického indukčního toku: $\epsilon=-\frac{d\Phi}{dt}$. Má-li sonda více závitů, je její citlivost na změnu magnetického pole dána součtem ploch jejích závitů $S_i$, což označujeme jako $A_\mathrm{eff}=\sum S_i$. Dále platí, že $\Phi=B\cdot A_\mathrm{eff}$. 

### MC na tokamaku GOLEM
Na tokamaku GOLEM jsou k dispozi čtyři Mirnovovy cívky umístěné na mezikruží ve stínu limiteru (viz následující obrázek), které se běžně využívají k určování vertikální a horitontální polohy. Každá z cívek má 91 závitů a jejich středy se nachází 93 mm od centra komory. Efektivní plocha každé z těchto cívek činí $A_\mathrm{eff}=3,8\cdot10^{-3}$ $\mathrm{m^{2}}$.

### Vnitřní Kvadrupól
Tokamak GOLEM také disponuje čtveřicí cívek o jednom závitu, které dohromady tvoří tzv. vnitřní kvadrupól. Ten byl v minulosti využíván k horizontální stabilizaci plazmatu, nicméně mi ho v rámci tohoto miniprojektu využijeme k měření průměrného vertikálního magnetického pole. Efektivní plocha je v tomto případě $A_\mathrm{eff}=0,83$ $\mathrm{m^{2}}$.


<table>
  <tr>
    <td valign="top"><img src="Fig/position_mc.jpg" width=80%></td>
  </tr>
</table>

### Načtení potřebných knihoven

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.font_manager as font_manager

from scipy import integrate, signal, interpolate
from scipy.io import loadmat

import pandas as pd
import requests


### Definování url adres odkud se mají brát data a požadovaná čísla výbojů

In [ ]:
BDdata_URL = "http://golem.fjfi.cvut.cz/shots/{shot_no}/Diagnostics/BasicDiagnostics/Results/{identifier}.csv"                  # Basic Diagnostics
MCdata_URL = "http://golem.fjfi.cvut.cz/shots/{shot_no}/Diagnostics/LimiterMirnovCoils/U_mc{identifier}.csv"                    # Mirnov Coils
FCdata_URL = "http://golem.fjfi.cvut.cz/shots/{shot_no}/Diagnostics/FastCameras/Camera_{identifier}/Camera{identifier}Position" # Fast Cameras
IQdata_URL = "http://golem.fjfi.cvut.cz/shots/{shot_no}/Devices/DASs/NI-squid/{identifier}.csv"                                 # Inner Quadrupole
PSdata_URL = "http://golem.fjfi.cvut.cz/shots/{shot_no}/Infrastructure/PositionStabilization/U^{identifier}_currclamp.csv"      # Position Stabilization

shot_no     = 49264 # number of selected shot
vacuum_shot = 49258 # number of vacuum shot with the same parameters as the selected shot_no (to elimate parasitic signal)


### Definování funkcí pro načtení dat z dané diagnostiky

In [ ]:
ds = np.DataSource(destpath='')  

def open_remote(shot_no, identifier, url_template):
    return ds.open(url_template.format(shot_no=shot_no, identifier=identifier))

def read_signal(shot_no, identifier, url, names=None, index_col = 0): 
    file = open_remote(shot_no, identifier, url)
    if names is None: 
        return pd.read_csv(file, index_col = index_col)
    else:    
        return pd.read_csv(file, names=names, index_col = index_col, header = None)
    
def smooth(data, win=11): 
    smoothed_data = signal.savgol_filter(data, win, 3)
    return smoothed_data    


### **Zpracování signálu**

#### **Odečtení offsetu**
Jelikož ještě před začátkem požadovaného měření sondy zaznamenávají jisté parazitní napětí (šum/offset), kvůli kterému nezačíná měření na nulové hodnotě, je nejdříve nutné tento offset odečíst. To lze provést odečtením průměru z několika hodnot signálu naměřeného sondou ještě před začátkem výboje.

#### **Integrace signálu**
Jak již bylo řečeno výše cívky zaznamenávají napětí indukované změnou, v našem případě, poloidálního magnetického pole generovaného plazmatem. K tomu abychom získali magnetické pole, je nutné naměřený signál zintegrovat a přenásobit příslušnou konstanou ($A_\mathrm{eff}$). Tedy: $B(t)=\frac{1}{A_\mathrm{eff}}\int_{0}^{t} U_\mathrm{sig}(\tau)d\tau$. 


#### **Eliminace parazitního signálu od $B_t$**
Dále je třeba uvážit, že cívky nemusí být (a také nejsou) umístěny přesně tak, aby jejich osy byly kolmé k toroidílnímu magnetickému poli. To má za následek, že získaný signál neodpovídá pouze poloidálnímu magnetickému poli, ale promítne se do něj parazitní signál od pole toroidálního, který je nutný eliminovat. 

K eliminaci využijeme data z vakuového výboje (bez napuštění pracovního plynu), případně výboje, u kterého nedošlo k průrazu plynu do plazmatu. V takovém případě totiž nemělo ani jak vzniknout pole poloidální a signál zaznamenaný cívkami tak odpovídá pouze parazitnímu signálu od toroidálního magnetického pole. Tento signál následně stačí od výbojů s plazmatem odečíst a získat tak požadované poloidální magnetické pole.

In [ ]:
def process_magnetic_measurements(raw_data, calibration_const, eliminate_vacuum=False, raw_data_vacuum=None, name=None, plot_elimination = True):
    """
    Process magnetic signal: removes offset, integrates, optionally subtracts vacuum (stray) field.
    
    Args:
        raw_data (pd.Series):        Measured magnetic signal
        const (float):               Scaling constant for integration.
        eliminate_vacuum (bool):     Whether to subtract vacuum background
        raw_data_vacuum (pd.Series): Corresponding vacuum signal (if eliminate_vacuum is True)
        name (str):                  Name for the resulting Series
    
    Returns:
        Integrated magnetic signal, optionally corrected for vacuum background (pd.Series)
    """
    
    # Clean raw signal
    data = raw_data.replace([np.inf, -np.inf, np.nan], 0)

    # Remove offset from early signal (pre-discharge)
    data -= data.loc[:0.9e-3].mean()

    # Integrate magnetic signal
    integrated_data = integrate.cumtrapz(data, x=data.index, initial=0) * calibration_const
    integrated_data = pd.Series(integrated_data, index=data.index * 1000, name=name)  # time in ms

    if eliminate_vacuum:
        if raw_data_vacuum is None:
            raise ValueError("raw_data_vacuum must be provided if eliminate_vacuum is True.")
        
        data_vacuum = raw_data_vacuum.replace([np.inf, -np.inf, np.nan], 0)
        data_vacuum -= data_vacuum.loc[:0.9e-3].mean()
        
        integrated_vacuum = integrate.cumtrapz(data_vacuum, x=data_vacuum.index, initial=0) * calibration_const
        integrated_vacuum = pd.Series(integrated_vacuum, index=data_vacuum.index * 1000, name=name + "_vacuum")
        
        # Subtract vacuum (stray field) contribution
        data_eliminated = integrated_data - integrated_vacuum
        data_eliminated.name = name + "_corrected"
        
        # ======== Plot elimination process ========== 
        # (for educational purposes)
        if plot_elimination:
            fig, ax = plt.subplots()
            integrated_vacuum.plot(label='vacuum data')
            integrated_data.plot(label='raw data')
            data_eliminated.plot(label='eliminated data')
            ax.legend()
            plt.title(name)
        
        return data_eliminated

    return integrated_data

### Magnetické pole měřené Mirnovovými cívkami a vnitřrním kvadrupólem

In [ ]:
# Vertical Magnetic Field Measured by Inner Quadrupole
# load data
dataNI = read_signal(shot_no, identifier = 'rawData', url=IQdata_URL, names=None)
U_InnerQuadr = dataNI['ch19'] # ch20 in plot on webpage but here indexing starts from 0
# process raw signal (elimination, integration,...)
B_InnerQuadr = process_magnetic_measurements(U_InnerQuadr,calibration_const = 1/0.83, eliminate_vacuum=False, name="B_InnerQuadr")*1e3 # T -> mT

# Local Magnetic Field Measured by four MCs
B_mc = pd.DataFrame()
# iterate over four MCs
for nMC in [1,9,5,13]:
    # load data
    U_mc = read_signal(shot_no, identifier = nMC, url=MCdata_URL, names=['Time', f'U_mc{nMC}'])
    U_mc = U_mc.sort_index()
    if vacuum_shot is not None:
        # eliminate stray magnetic fields using a vacuum shot with similar parameters
        U_mc_vac = read_signal(vacuum_shot, identifier = nMC, url=MCdata_URL, names=['Time', f'U_mc{nMC}'])
        U_mc_vac = U_mc_vac.sort_index()
        B_mc[f'B_mc{nMC}'] = process_magnetic_measurements(U_mc.iloc[:, 0],calibration_const = 1/(3.8e-3), eliminate_vacuum=True, name=f"B_mc{nMC}",
                                                           raw_data_vacuum=U_mc_vac.iloc[:, 0], plot_elimination=False)*1e3 # T -> mT
    else:
        # use data w/o elimination of stray mag. field
        B_mc[f'B_mc{nMC}'] = process_magnetic_measurements(U_mc.iloc[:, 0],calibration_const = 1/(3.8e-3), eliminate_vacuum=False, name=f"B_mc{nMC}")*1e3 # T -> mT
    

### Vykreslení zpracovaných signálů od jednotlivých magnetikých diagnostik

In [ ]:

fig, ax = plt.subplots(1,3, figsize=(12,3))
# plot signals measured by mc1(LFS) and mc9(HFS) ~ local vertical mag. field Bz 
B_mc['B_mc1'].plot(ax=ax[0], label = 'mc1')
B_mc['B_mc9'].plot(ax=ax[0], label = 'mc9')
ax[0].set(xlabel = 'Time [ms]', ylabel = '$B_z$ [mT]')

# plot signals measured by mc5(top) and mc13(bottom) ~ local horizontal mag. field Br 
B_mc['B_mc5'].plot(ax=ax[1], label = 'mc5')
B_mc['B_mc13'].plot(ax=ax[1], label = 'mc13')
ax[1].set(xlabel = 'Time [ms]', ylabel = '$B_r$ [mT]')

# plot signals measured by Inner Quadrupole ~ mean vertical mag. field Bz 
B_InnerQuadr.plot(ax=ax[2], label = 'Inner Quadrupole')
ax[2].set(xlabel = 'Time [ms]', ylabel = '$B_z$ [mT]')

for ax_i in ax.flatten():
    ax_i.grid(True)
    ax_i.legend()
    
plt.tight_layout()


### Začátek a konec výboje

In [ ]:
plasma_start = float(requests.get(f'http://golem.fjfi.cvut.cz/shots/{shot_no}/Diagnostics/PlasmaDetection/Results/t_plasma_start').text)
plasma_end = float(requests.get(f'http://golem.fjfi.cvut.cz/shots/{shot_no}/Diagnostics/PlasmaDetection/Results/t_plasma_end').text)

print ('Plasma start =', round(plasma_start, 3), 'ms')
print ('Plasma end =', round(plasma_end, 3), 'ms')


## Výpočet polohy plazmatu
Pro výpočet polohy plazmatu využijeme přiblížení rovného vodiče a tedy zanedbáme v našem výpočtu toroidální efekty. Magnetické pole generované přímým vodičem je dáno vztahem $B(r)=\frac{\mu_0 I_p}{2\pi r}$. 

Uvažme nyní kladný vertikální posun $\Delta z>0$ středu plazmatu z centra komory směrem nahoru. Potom velikost poloidálního magnetického pole naměřená cívkou $mc_5$ je dána předpisem: $B_{mc_5}=\frac{\mu_0 I_p}{2\pi (r-\Delta z)}$ a cívkou $mc_{13}$ předpisem: $B_{mc_{13}}=\frac{\mu_0 I_p}{2\pi (r+\Delta z)}$. Z toho již, provedením algebraických úprav a dosazením $b$ za $r$, pro velikost vertikálního posuvu dostáváme $\Delta z=\frac{B_{mc_5}-B_{mc_{13}}}{B_{mc_5}+B_{mc_{13}}}\cdot b$, kde $b=93$ $\mathrm{mm}$. 

Analogicky lze dospět i ke vztahu pro změnu radiální polohy $\Delta r=\frac{B_{mc_1}-B_{mc_9}}{B_{mc_1}+B_{mc_9}}\cdot b$.

### Radiální poloha plazmatu $\Delta r$
$\Delta r=\frac{B_{mc_1}-B_{mc_9}}{B_{mc_1}+B_{mc_9}}\cdot b$, kde $b=93$ $\mathrm{mm}$

<div class="alert alert-block alert-info">

**ÚKOL:** Doplňte následující funkci tak, aby vracela změnu radiální polohy středu středu plazmatu $\Delta r$ (i s časem -> formát pd.Series).
  
    
Pozn. Sběr dat probíhal po mnohem delší dobu než je délka výboje => signál je vhodné "oříznout", tj. místo *return r* lze použít *return r.loc[plasma_start:plasma_end]*.
    
</div> 

In [ ]:
def horpos(Bmc1, Bmc9):    
    """
    Calculate horizontal plasma position based on Mirnov coil measurements.

    Parameters:
        Bmc1 (pd.Series): Magnetic field signal from Mirnov coil #1.
        Bmc9 (pd.Series): Magnetic field signal from Mirnov coil #9.

    Returns:
        pd.Series: Horizontal plasma position (r) in millimeters.
    """
    
    
    #r = ....
    r = r.replace([np.nan], value = 0)
    
    return r.loc[plasma_start:plasma_end]
    

In [ ]:
# Calculate horizontal plasma position
r = horpos(B_mc['B_mc1'], B_mc['B_mc9'])

# Plot horizontal plasma position (displacement of the plasma axis from the chamber center)
fig,ax=plt.subplots()
ax = r.plot()
ax.set(ylim=(-85,85), xlim=(plasma_start,plasma_end), xlabel= 'Time [ms]', ylabel = '$\Delta$r [mm]', title = 'Radial plasma position #{}'.format(shot_no))
ax.axhline(y=0, color='k', ls='--', lw=1, alpha=0.4)

### Vertikální poloha plazmatu $\Delta z$
$z=\frac{B_{mc_5}-B_{mc_{13}}}{B_{mc_5}+B_{mc_{13}}}\cdot b$, kde $b=93$ $\mathrm{mm}$

<div class="alert alert-block alert-info">

**ÚKOL:** Doplňte následující funkci tak, aby vracela změnu vertikální polohy středu středu plazmatu $\Delta z$ (i s časem -> formát pd.Series) 
    
</div> 

In [ ]:
def verpos(Bmc5, Bmc13):
    """
    Calculate vertical plasma position based on Mirnov coil measurements.

    Parameters:
        Bmc5  (pd.Series): Magnetic field signal from Mirnov coil #5.
        Bmc13 (pd.Series): Magnetic field signal from Mirnov coil #13.

    Returns:
        pd.Series: Vertical plasma position (z) in millimeters.
    """
    
    
    return z.loc[plasma_start:plasma_end]

<div class="alert alert-block alert-info">

**ÚKOL:** Vykreslete změnu vertikální polohy $\Delta z$ v závislosti na čase. 
    (Inspirujte se radiální polohou.)
    
</div> 

In [ ]:
# Calculate vertical plasma position
# ...
# Plot vertical plasma position (displacement of the plasma axis from the chamber center)
fig, ax = plt.subplots()
# ...

### Poloměr plazmatu $a$ 
Na tokamaku GOLEM je velikost plazmatického prstence omezena kruhovou clonou tzv. limiterem, umístěným na pozici $r=85$ mm. 

<div class="alert alert-block alert-info">

**ÚKOL:** Na základě znalosti změny vertikální polohy $\Delta z$, radiální polohy $\Delta r$ a polohy limiteru (limituje/ořezává prstenec plazmatu) odvoďte vzorec pro výpočet poloměru plazmatu $a$.
    
</div> 

<div class="alert alert-block alert-warning">

**Vzorec pro výpočet poloměru plazmatu:** $a = $
    
</div> 

<div class="alert alert-block alert-info">

**ÚKOL:** Nyní tento vzorec využijte a napište funkci vracející poloměr plazmatu (i s časem -> formát pd.Series).
    
</div> 

In [ ]:
def plasma_radius(B_mc):
    
    #a = ...
    a = a.replace([np.nan], value = 0)
    return a.loc[plasma_start:plasma_end]

<div class="alert alert-block alert-info">

**ÚKOL:** Vykreslete poloměr plazmatu $a$ v závislosti na čase. 
    
</div> 

In [ ]:

fig, ax = plt.subplots()


## Poloha Plazmatu - Rychlé Kamery
Tokamak GOLEM aktuálně disponuje dvěma rychlými barevnými kamerami Photron Mini UX100. Z naměřeného signálu pak lze určit polohu plazmatu, která v porovnání s MC není ovlivněna parazitním magnetickým polem.

<table>
  <tr>
    <td valign="top"><img src="Fig/setup.jpg" width=60%></td>
    <td valign="top"><img src="Fig/miniUX50.png" width=40%></td>
  </tr>
</table>

In [ ]:
# Load horizontal and vertical position calculated from measurements of fast cameras
r_camera = read_signal(shot_no, identifier = 'Radial', url=FCdata_URL, names=['Time', 'Radial'])
z_camera = read_signal(shot_no, identifier = 'Vertical', url=FCdata_URL, names=['Time', 'Vertical'])

# Plot 
fig, ax = plt.subplots(2,1, sharex=True)
r_camera.plot(ax=ax[0])
z_camera.plot(ax=ax[1])
ax[0].set(ylim=(-85,85), xlim=(plasma_start,plasma_end), xlabel= 'Time [ms]', ylabel = '$\Delta r$ [mm]', title = f'Plasma position - Fast Cameras #{shot_no}')
ax[0].axhline(y=0, color='k', ls='--', lw=1, alpha=0.4)
ax[1].set(ylim=(-85,85), xlim=(plasma_start,plasma_end), xlabel= 'Time [ms]', ylabel = '$\Delta z$ [mm]', title = '')
ax[1].axhline(y=0, color='k', ls='--', lw=1, alpha=0.4)
plt.tight_layout()

### Porovnání Polohy Plazmatu

In [ ]:
# Import function from external .py file
import compare_fc_vs_mc as fc_vs_mc
fig_r, fig_z = fc_vs_mc.compare_fc_mc(shot_no, mode = 'advance')


## **Stabilizace Plazmatu**
<table>
  <tr>
    <td valign="top"><img src="Fig/StabScheme.jpg" width=60%></td>
  </tr>
</table>

<table>
  <tr>
    <td valign="top"><img src="Fig/field_orientation_white.png" width=100%></td>
    <td valign="top"><img src="Fig/HorizontalField.png" width=90%></td>
    <td valign="top"><img src="Fig/VerticalField.png" width=90%></td>
  </tr>
</table>



In [ ]:
def create_data_stab(shot_no):
    """
    Load and process stabilization coil current signals for a given shot

    Args:
        shot_no (int): Shot number

    Returns:
        pd.DataFrame: Time-indexed DataFrame with processed currents:
            - I_radialStab [A]
            - I_verticalStab [A]
    """
    
    
    c_currentClamp = 1/0.05 # calibration constant for converting voltage to amps
    data_stab = pd.DataFrame()
    data_stab = pd.concat([read_signal(shot_no, identifier = pos, url=PSdata_URL, names=[f'I_{pos}Stab'], index_col = None)*c_currentClamp 
                           for pos in ['radial','vertical']], axis = 'columns');


    # get oscilloscope's time
    dt = float(requests.get(f'http://golem.fjfi.cvut.cz/shotdir/{shot_no}/Devices/Oscilloscopes/RigolMSO5204-a/ScopeSetup/XINC').text)*1e6
    data_stab.index = pd.Series(np.linspace(0, dt, data_stab.shape[0])).rename('Time')
    
    # smooth signals
    data_stab['I_radialStab']   = smooth(data_stab['I_radialStab'])
    data_stab['I_verticalStab'] = smooth(data_stab['I_verticalStab'])
    
    return data_stab



def make_quiver_plot(ax, B_stab, x, y, title_plot=None, plot_type='log', step=4, sc = 1):
    """
    Plot a quiver plot of magnetic field B_stab on a coarser mesh defined by `step`.
    
    Args:
        ax         : matplotlib Axes
        B_stab     : object with .Br_sum and .Bz_sum (2D arrays)
        x, y       : 1D arrays (original meshgrid axes)
        title_plot : str, optional
        plot_type  : 'log' or 'std'
        step       : int, step size to reduce resolution
        sc         : scaling (e.g. use for opossite current direction)

    Returns:
        ax
        
    """
    plot_type = plot_type.lower().strip() # make matching more robust

    Br = B_stab.num.Br_sum*sc
    Bz = B_stab.num.Bz_sum*sc

    X, Y = np.meshgrid(x, y, indexing='ij')
    points = np.column_stack((X.ravel(), Y.ravel()))
    Br_vals = Br.ravel()
    Bz_vals = Bz.ravel()

    # Create a less dense grid
    x_coarse = x[::step]
    y_coarse = y[::step]
    Xc, Yc = np.meshgrid(x_coarse, y_coarse, indexing='ij')
    grid_points = np.column_stack((Xc.ravel(), Yc.ravel()))

    # Interpolate to coarser grid
    Br_interp = interpolate.griddata(points, Br_vals, grid_points, method='linear').reshape(Xc.shape)
    Bz_interp = interpolate.griddata(points, Bz_vals, grid_points, method='linear').reshape(Xc.shape)

    # Compute magnitude
    magnitude = np.sqrt(Br_interp**2 + Bz_interp**2)
    magnitude[magnitude == 0] = 1e-9 # avoid division by zero

    if 'log' in plot_type:
        log_magnitude = np.log10(magnitude + 1e-5)
        Br_unit = Br_interp / magnitude
        Bz_unit = Bz_interp / magnitude

        q=ax.quiver(Xc, Yc, Br_unit, Bz_unit, log_magnitude,
                  cmap='viridis', scale_units='xy', scale=40)
        fig = ax.get_figure()
        cbar = fig.colorbar(q, ax=ax)
        
    elif 'std' in plot_type:
        ax.quiver(Xc, Yc, Br_interp, Bz_interp, color='tab:blue', scale=1e3)
        
    else:
        raise ValueError("Unsupported plot_type: must be 'log' or 'std'")

    
    ax.plot(B_stab.coord.R, B_stab.coord.Z,
         color="#D95319", linestyle='none', marker='.', linewidth=0.3, label='')

    circle = patches.Circle((0.4, 0.0), 0.085, edgecolor='k', facecolor='none', linestyle='-', label='limiter', linewidth=1.6)
    ax.add_patch(circle)
    
    ax.set_title(title_plot)
    ax.set_aspect('equal')
    
    
    return ax


In [ ]:
# Load stabilization coil current
data_stab = create_data_stab(shot_no)

# ==============================================================
# Load Pre-calculated Mag. Field Generated by Stab. Coils
# ==============================================================

# Load .mat file with pre-calculated mag. field
stab_matlab = loadmat('Bcoils_stabilization.mat', struct_as_record=False, squeeze_me=True)
Bcoils      = stab_matlab['Bcoils']

# Extract meshgrid
x = Bcoils.meshgrid.x  # radial coordinate
y = Bcoils.meshgrid.y  # vertical coordinate

fig, ax= plt.subplots(1,2, figsize = (12,6))
ax[0] = make_quiver_plot(ax[0], Bcoils.HorStab, x, y, title_plot = 'Horizontal Position Stabilization',step = 3)
ax[1] = make_quiver_plot(ax[1], Bcoils.VertStab, x, y, title_plot = 'Vertical Position Stabilization',step = 3)

for ax_i in ax.flatten():
    ax_i.set(xlabel='R [m]', ylabel = 'Z [m]')

# Porovnání Vybraných Výbojů

In [ ]:
def create_tab_mc(shot_no, vacuum_shot):
    """
    Creates a DataFrame containing the radial position, vertical position, and plasma radius 
    calculated from magnetic (Mirnov) coil measurements.

    Args:
        shot_no (int):            Discharge number.
        vacuum_shot (int | None): Reference vacuum shot (for stray field subtraction). 
                                  If None, stray fields are not subtracted.

    Returns:
        pd.DataFrame: DataFrame with columns:
            - r_{shot_no} : Radial position [mm]
            - z_{shot_no} : Vertical position [mm]
            - a_{shot_no} : Plasma radius [mm]
    """
    B_mc = pd.DataFrame()

    # Iterate over four magnetic coils (MCs)
    for nMC in [1, 9, 5, 13]:
        # Load main signal
        U_mc = read_signal(shot_no, identifier=nMC, url=MCdata_URL, names=['Time', f'U_mc{nMC}'])
        U_mc = U_mc.sort_index()

        if vacuum_shot is not None:
            # Load corresponding vacuum shot signal
            U_mc_vac = read_signal(vacuum_shot, identifier=nMC, url=MCdata_URL, names=['Time', f'U_mc{nMC}'])
            U_mc_vac = U_mc_vac.sort_index()

            # Process signal with stray field elimination
            B_mc[f'B_mc{nMC}'] = process_magnetic_measurements(raw_data          = U_mc.iloc[:, 0],
                                                               calibration_const = 1 / (3.8e-3),
                                                               raw_data_vacuum   = U_mc_vac.iloc[:, 0],
                                                               name              = f"B_mc{nMC}",
                                                               eliminate_vacuum  = True,
                                                               plot_elimination  = False) * 1e3  # Convert T to mT
        else:
            # Process signal without stray field elimination
            B_mc[f'B_mc{nMC}'] = process_magnetic_measurements(raw_data          = U_mc.iloc[:, 0],
                                                               calibration_const = 1 / (3.8e-3),
                                                               name              = f"B_mc{nMC}",
                                                               eliminate_vacuum  = False) * 1e3  # Convert T to mT

    # Calculate position and size
    r = horpos(B_mc['B_mc1'], B_mc['B_mc9'])
    z = verpos(B_mc['B_mc5'], B_mc['B_mc13'])
    a = plasma_radius(B_mc)

    # Combine into a single DataFrame with shot-specific suffixes
    df = pd.concat([r.rename('r'), z.rename('z'), a.rename('a')], axis='columns').add_suffix(f'_{shot_no}')
    
    return df


def create_tab_cameras(shot_no):
    """
    Creates a DataFrame containing the radial and vertical position of the plasma axis 
    estimated from fast camera diagnostics.

    Args:
        shot_no (int): Discharge number.

    Returns:
        pd.DataFrame: DataFrame with columns:
            - Radial_{shot_no}  : Radial plasma position [mm]
            - Vertical_{shot_no}: Vertical plasma position [mm]
    """
    r_camera = read_signal(shot_no, identifier='Radial', url=FCdata_URL, names=['Time', 'radial'])
    z_camera = read_signal(shot_no, identifier='Vertical', url=FCdata_URL, names=['Time', 'vertical'])

    df = pd.concat([r_camera, z_camera], axis='columns').add_suffix(f'_{shot_no}')
    
    return df


In [ ]:
def compare_discharges(shots, vacuum_shots, diagnostic='MC'):
    """
    Compare selected discharges using magnetic coils or camera diagnostics.
    
    Args:
        shots (list of int): List of discharge shot numbers.
        vacuum_shots (list of int): List of corresponding vacuum shot numbers (for background subtraction).
        diagnostic (str): Type of diagnostic to use: 'MC' (Mirnov Coils) or 'FC' (Fast Cameras).
    """
    
    if 'MC' in diagnostic:
        df_mc = pd.DataFrame()
        # Load and combine Mirnov Coils data for each shot
        for shot_no, vacuum_shot in zip(shots, vacuum_shots):
            df_mc = pd.concat([df_mc, create_tab_mc(shot_no, vacuum_shot)])

    # Load and combine Fast Cameras data if needed
    elif 'FC' in diagnostic:
        df_cameras = pd.DataFrame()
        for shot_no in shots:
            df_cameras = pd.concat([df_cameras, create_tab_cameras(shot_no)])
             

    # Initialize plot: 3 rows x 1 column
    fig, ax = plt.subplots(3, 1, figsize=(10, 8), dpi=70, sharex=True)
    fontname = 'DejaVu Sans'
    fontsize = 15

    # Signal names and colors
    symbols  = ['r','z'] 
    position = ["radial", "vertical"]
    colors   = ['tab:blue', 'tab:red', 'tab:orange', 'tab:green', 'tab:purple', 'tab:cyan', 'tab:olive']

    for shot_no, color in zip(shots, colors):
        # Plot mirnov coil data
        if 'MC' in diagnostic:
            t_start = float(requests.get(f'http://golem.fjfi.cvut.cz/shots/{shot_no}/Diagnostics/PlasmaDetection/Results/t_plasma_start').text)
            t_end   = float(requests.get(f'http://golem.fjfi.cvut.cz/shots/{shot_no}/Diagnostics/PlasmaDetection/Results/t_plasma_end').text)
            
            for i in range(2):
                df_mc[f'{symbols[i]}_{shot_no}'].plot(ax=ax[i], label=f'#{shot_no}', color=color)

        # Plot fast camera data
        elif 'FC' in diagnostic:
            for i in range(2):
                series = df_cameras[df_cameras[f'{position[i]}_{shot_no}'].notna()][f'{position[i]}_{shot_no}']
                series.plot(ax=ax[i], label=f'#{shot_no}', color=color)

        # Plot stabilization coil currents
        data_stab = create_data_stab(shot_no)
        data_stab['I_verticalStab'].plot(ax=ax[2], label=f'Vertical #{shot_no}', color=color, linestyle='-', linewidth=1.6)
        data_stab['I_radialStab'].plot(ax=ax[2], label=f'Horizontal #{shot_no}', color=color, linestyle='-.', linewidth=1.6)

    # === Axis formatting ===
    for i in range(3):
        if i != 2:
            ax[i].set_ylabel(f'$\Delta${symbols[i]} [mm]', fontname=fontname, fontweight='bold', fontsize=fontsize)
            ax[i].axhline(y=0, color='k', linestyle='--', linewidth=1, alpha=0.4)
            ax[i].set_ylim(-85, 85)
        else:
            ax[i].set_ylabel('I [A]', fontname=fontname, fontweight='bold', fontsize=fontsize)
            ax[i].set_xlabel('Time [ms]', fontname=fontname, fontweight='bold', fontsize=fontsize)

        ax[i].set_xlim(1.5, 23)
        ax[i].grid(which='major', color='gray', linestyle='solid', linewidth=0.5)
        ax[i].grid(which='minor', color='gray', linestyle='dashed', linewidth=0.3)
        ax[i].tick_params(labelsize=12)

        # Apply bold font to tick labels
        ticks_font = font_manager.FontProperties(family=fontname, size=12, weight='bold')
        for label in ax[i].get_xticklabels() + ax[i].get_yticklabels():
            label.set_fontproperties(ticks_font)

        # Format legend
        legend = ax[i].legend(loc='best', shadow=True, fancybox=False)
        legend.get_frame().set_linewidth(1)
        legend.get_frame().set_edgecolor('k')
        for text in legend.get_texts():
            text.set_fontname(fontname)
            text.set_fontsize(fontsize)
        for line, text in zip(legend.get_lines(), legend.get_texts()):
            text.set_color(line.get_color())

    plt.tight_layout()
    
    # fig.savefig('compare_discharges.png')  # Uncomment to save the figure
    return

In [ ]:
shots        = [49263, 49264] # selected shots
vacuum_shots = [49258, 49258] # corresponding vacuum shots

compare_discharges(shots, vacuum_shots, 'FC')

## **Bonus**: Polohová Stabilizace a Breakdown 
<table>
  <tr>
    <td valign="top"><img src="Fig/field_orientation_white.png" width=100%></td>
    <td valign="top"><img src="Fig/breakdown_strayField.jpg" width=50%></td>
  </tr>
</table>

In [ ]:
#Import function for use in comparison during breakdown